In [1]:
import pandas as pd, numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv('./input/input.csv')

In [3]:
data.describe()

,contribution,contest_count,rating,max_rating,friends_count,problem_count,avg_difficulty,avg_difficulty20,median,tag_combinatorics,...,difficulty_bucket3,difficulty_bucket4,difficulty_bucket5,difficulty_bucket6,difficulty_bucket7,difficulty_bucket8,difficulty_bucket9,difficulty_bucket10,difficulty_bucket11,difficulty_bucket12
count,6441.000000,6441.000000,6441.000000,6441.000000,6441.000000,6441.000000,6441.000000,6441.000000,6441.000000,6441.000000,...,6441.000000,6441.000000,6441.000000,6441.000000,6441.000000,6441.000000,6441.000000,6441.000000,6441.0,6441.000000
mean,0.648968,20.576619,1440.101848,1556.414687,28.782487,127.123428,1206.016621,1629.946308,1181.183046,3.465766,...,18.191430,15.219531,7.868964,4.616053,2.162552,1.292656,0.574755,0.152150,0.0,0.022667
std,5.745607,20.917052,232.426607,182.517250,70.549313,154.928163,167.440872,282.403082,177.788077,6.573742,...,24.797218,24.712344,16.140394,13.114247,7.917106,5.984573,2.908497,1.113085,0.0,0.259805
min,-43.000000,1.000000,603.000000,1366.000000,0.000000,1.000000,500.000000,500.000000,500.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,0.000000,5.000000,1287.000000,1413.000000,4.000000,25.000000,1092.592593,1443.589744,1100.000000,0.000000,...,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
50%,0.000000,14.000000,1414.000000,1488.000000,11.000000,71.000000,1192.391304,1607.407407,1200.000000,1.000000,...,8.000000,5.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
75%,0.000000,28.000000,1581.000000,1659.000000,29.000000,171.000000,1301.111111,1794.444444,1300.000000,4.000000,...,26.000000,19.000000,7.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000
max,151.000000,163.000000,2500.000000,2578.000000,1657.000000,1611.000000,2013.186813,2716.666667,2000.000000,74.000000,...,233.000000,300.000000,232.000000,201.000000,162.000000,132.000000,81.000000,26.000000,0.0,8.000000


In [4]:
data = pd.concat([data, pd.get_dummies(data["organization"],prefix='organization')], axis = 1)
data.drop(["organization"], axis = 1, inplace = True)
data = shuffle(data)
data_columns = list(data.columns)
data_columns.remove("rating")
data_columns.remove("country")
data_columns.remove("handle")
data_columns.remove("max_rating")
data_columns.remove("rank")
train, test = train_test_split(data, test_size = 0.3)
x_train = train[data_columns]
y_train = train["max_rating"]
x_test = test[data_columns]
y_test = test["max_rating"]

In [5]:
x_train.head()

,contribution,contest_count,friends_count,problem_count,avg_difficulty,avg_difficulty20,median,tag_combinatorics,tag_constructive algorithms,tag_graphs,...,organization_National Institute of Technology Patna,"organization_Netaji Subhas Institute of Technology, Delhi",organization_Nirma University,organization_Pune Institute of Computer Technology,organization_SRM University,organization_SSN College Of Engineering,organization_SVNIT,organization_Thapar University,organization_University School of Information Technology,"organization_Vellore Institute of Technology,Vellore"
2529,0,16,12,53,1224.528302,1670.000000,1200,0,10,1,...,0,0,0,0,0,0,0,0,0,0
4143,0,23,60,187,1431.550802,1956.756757,1400,7,26,13,...,0,0,0,0,0,0,0,0,0,0
4023,0,23,0,69,995.652174,1338.461538,900,2,6,0,...,0,1,0,0,0,0,0,0,0,0
3312,0,4,4,6,1200.000000,1600.000000,1000,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5364,0,22,12,229,1214.847162,1640.000000,1200,4,21,11,...,0,0,0,1,0,0,0,0,0,0


In [6]:
def LinearRegressor(x,y):
    return LinearRegression().fit(x, y)
def RandomForest(x,y):
    return RandomForestRegressor(n_estimators = 1200, min_samples_leaf = 6, n_jobs = -1, verbose = 1).fit(x, y)
def XGB(x,y):
    return XGBRegressor().fit(x,y)

In [7]:
predictions = np.zeros((len(test), 1))
model = XGB(x_train, y_train)
predictions = model.predict(x_test).reshape(-1, 1)
labels = y_test.values.reshape(-1, 1)
# model.score(x_test, labels)

error = mean_squared_error(y_test, predictions)**0.5
print(error)

80.78592460878582


In [8]:
x = np.hstack([predictions,labels])

In [9]:
pd.DataFrame(x).to_csv("output.csv")